<a href="https://colab.research.google.com/github/interritus141/COMP0031-Group-Research-Project/blob/master/COMP0031.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

In [ ]:
import gym
from stable_baselines3 import A2C, DDPG, DQN, PPO
from ta import add_all_ta_features
import yfinance as yf
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import torch

import functools
import os
import csv
import json
from math import prod

# Data

## Tech Indicators

In [ ]:
def add_ta(df):
  ta_df = add_all_ta_features(df, open="Open", high="High", low="Low", close="Close", volume="Volume")
  # print(ta_df.columns)
  ta_df = ta_df[["Open", "High", "Low", "Close", "Volume", "Adj Close", "volume_obv",
                                "volume_adi", "trend_adx", "momentum_ao", "trend_macd", "momentum_rsi", 
                                "momentum_stoch"]]
  ta_df = ta_df.fillna(ta_df.mean())
  return ta_df

## Stocks

1. Apple Inc. (AAPL)
2. Microsoft Corp. (MSFT)
3. Amazon.com, Inc. ( AMZN)
4. Tesla, Inc. (TSLA)
5. Nvidia Corp. (NVDA)

In [ ]:
# interval = 1m,2m,5m,15m,30m,60m,90m,1h,1d,5d,1wk,1mo,3mo
# prepost = T/F

stock_volatilities = {}
stocks = dict.fromkeys(["AAPL", "MSFT", "AMZN", "TSLA", "NVDA", "CAAS"])

for stock in stocks.keys():
  stock_df = yf.download(stock, start="2018-01-01", end="2022-12-31", keepna=True)
  stock_df = stock_df.fillna(stock_df.mean())
  stock_df = add_ta(stock_df)
  stocks[stock] = stock_df
  stock_volatilities[stock] = None

## Volatility

In [ ]:
def add_volatility(df, df_name):
  df["Log returns"] = np.log(df['Close'] / df['Close'].shift())
  stock_volatilities[df_name] = df['Log returns'].std() * 252 ** .5

In [ ]:
def visualise_volatility(df, df_name, volatility):
  fig, ax = plt.subplots()
  df['Log returns'].hist(ax=ax, bins=50, alpha=0.6, color='b')
  ax.set_xlabel("Log return")
  ax.set_ylabel("Freq of log return")
  ax.set_title("{:s} volatility: {:.2f}%".format(df_name, volatility*100))

for stock, stock_df in stocks.items():
  add_volatility(stock_df, stock)
  # visualise_volatility(stock_df, stock, stock_volatilities[stock]) # use to generate graphs

print(stock_volatilities)

## Cov

In [ ]:
def add_cov(df):
  df = df.reset_index()

  cov_list = []
  return_list = []

  # look back is one year
  lookback=252
  for i in range(lookback,len(df.index.unique())):
    data_lookback = df.iloc[i-lookback:i,:]
    price_lookback=data_lookback.pivot_table(index = 'Date', values = 'Close')
    return_lookback = price_lookback.pct_change().dropna()
    return_list.append(return_lookback)

    covs = return_lookback.cov().values 
    cov_list.append(covs)


  df_cov = pd.DataFrame({'Date':df["Date"].unique()[lookback:],'cov_list':cov_list,'return_list':return_list})
  df = df.merge(df_cov, on='Date')
  df = df.sort_values(['Date']).reset_index(drop=True)
  return df

In [ ]:
# high volatility
new_aapl_df = add_cov(stocks["AAPL"])

# low volatility
new_tsla_df = add_cov(stocks["TSLA"])

In [ ]:
data_aapl_df = new_aapl_df.copy()
data_aapl_df["tic"] = "AAPL"

data_tsla_df = new_tsla_df.copy()
data_tsla_df["tic"] = "TSLA"

mixed_df = pd.concat([data_aapl_df, data_tsla_df])

# Environment

## Agent

In [ ]:
AGENT_STR_TO_OBJECT = {
    "A2C": A2C,
    "DDPG": DDPG,
    "PPO": PPO,
}
policy = "MlpPolicy"

In [ ]:
TYPE_TO_NAMES = {
    "A2C": ["A2C1", "A2C2", "A2C3"],
    "DDPG": ["DDPG1", "DDPG2", "DDPG3"],
    "PPO": ["PPO1", "PPO2", "PPO3"],
    "Mixed": ["A2C", "DDPG", "PPO"],
}

## Competitive

In [ ]:
class CompetitivePMEnv(gym.Env):
  metadata = {"render_modes": ["human"], "name": "marlpm_v1"}

  def __init__(
      self, 
      df,
      stock_dim,
      hmax,
      initial_amount,
      transaction_cost_pct,
      reward_scaling,
      state_space,
      action_space,
      tech_indicator_list,
      turbulence_threshold=None,
      lookback=252,
      day=0,
      render_mode=None,
      algo_type="Mixed", # default=mixed
  ):

    # attributes
    self.lookback=lookback
    self.df = df
    self.stock_dim = stock_dim
    self.hmax = hmax
    self.initial_amount = initial_amount
    self.transaction_cost_pct =transaction_cost_pct
    self.reward_scaling = reward_scaling
    self.state_space = state_space
    self.action_dim = action_space
    self.tech_indicator_list = tech_indicator_list
    self.possible_agents = TYPE_TO_NAMES[algo_type]
    
    # buy/sell ratio reference, to explore
    self.end_day = len(self.df.index.unique()) - 1
    self.stock_volume_reference = 10000

    # spaces
    # check: spaces for observations only? sharing will affect?
    self.action_space = gym.spaces.Box(low = -1, high = 1, shape = (self.action_dim,))
    self.observation_space = gym.spaces.Box(low=0, high=np.inf, shape = (1+len(self.tech_indicator_list), self.state_space))

    # agents
    self.agent_name_mapping = {
        # agent: AGENT_STR_TO_OBJECT[agent](policy, self) for agent in self.possible_agents
        agent: A2C(policy, self, n_steps=self.end_day) for agent in self.possible_agents
    }
    self.training_agent = None
    self.day = {
        agent: day for agent in self.possible_agents
    }

    # data
    self.data = {
        agent: self.df.loc[self.day[agent],:] for agent in self.possible_agents
    }
    self.covs = {
        agent: [[x[0][0] for x in self.data[agent]['cov_list']]] for agent in self.possible_agents
    }
    self.state = {
        agent: np.append(np.array(self.covs[agent]), [self.data[agent][tech].values.tolist() for tech in self.tech_indicator_list ], axis=0) for agent in self.possible_agents
    }
    self.terminal = False     
    self.turbulence_threshold = turbulence_threshold   

    # memory
    self.portfolio_value = {
        agent: self.initial_amount for agent in self.possible_agents
    }
    self.asset_memory = {
        agent: [self.initial_amount] for agent in self.possible_agents
    }
    self.portfolio_return_memory = {
        agent: [0] for agent in self.possible_agents
    }
    # stock ratio
    self.actions_memory = {
        agent: [[0]*self.stock_dim] for agent in self.possible_agents
    }
    self.date_memory = {
        agent: [self.data[agent]["Date"].unique()[0]] for agent in self.possible_agents
    }

    # free cash
    self.money_memory = {
        agent: [self.initial_amount] for agent in self.possible_agents
    }
    # cash + stock value
    self.total_value_memory = {
        agent: [self.initial_amount] for agent in self.possible_agents
    }
    # individual actions collection
    self.individual_preds = {
        agent: [] for agent in self.possible_agents
    }

    # render mode
    self.render_mode = render_mode

  def collect_individual_preds(self):
    self.individual_preds = {
        agent: [] for agent in self.possible_agents
    }
    for i in range(self.end_day+1):
      # states are somewhat static
      curr_data = self.df.loc[i,:]
      curr_covs = [[x[0][0] for x in curr_data['cov_list']]]
      curr_state = np.append(np.array(curr_covs), [curr_data[tech].values.tolist() for tech in self.tech_indicator_list ], axis=0)
      for agent in self.possible_agents:
        action, _states = self.agent_name_mapping[agent].predict(curr_state)
        self.individual_preds[agent].append(action)
  
  def step(self, actions):
    # print(self.day)
    # print(actions)

    # termination
    # check: termination determined correctly?
    self.terminal = self.day[self.training_agent] >= self.end_day
    # self.terminal = {
    #     agent: self.day >= len(self.df.index.unique())/self.stock_dim-1 for agent in self.possible_agents
    # }

    if self.terminal:
      df = pd.DataFrame(self.portfolio_return_memory[self.training_agent])
      df.columns = ['daily_return']

      print("=================================")
      print("begin_total_asset:{}".format(self.asset_memory[self.training_agent][0]))           
      print("end_total_asset:{}".format(self.portfolio_value[self.training_agent]))

      df_daily_return = pd.DataFrame(self.portfolio_return_memory[self.training_agent])
      df_daily_return.columns = ['daily_return']
      if df_daily_return['daily_return'].std() !=0:
        sharpe = (252**0.5)*df_daily_return['daily_return'].mean()/ \
                  df_daily_return['daily_return'].std()
        print("Sharpe: ",sharpe)
      print("=================================")

      return self.state[self.training_agent], self.reward[self.training_agent], self.terminal, {}

    else:
      # loop through all agents so that each of them predict an action (portfolio weights)
      for agent in self.possible_agents:
        # get action
        if agent == self.training_agent:
          action = actions
        else:
          # action, _states = self.agent_name_mapping[agent].predict(self.state[agent], deterministic=False)
          action = self.individual_preds[agent][self.day[agent]]

        # normalisation
        weights = self.softmax_normalization(action) 

        # stock ratio - buy/sell/hold
        prev_stock_ratio = self.actions_memory[agent][-1]
        diff_stock_ratio = prev_stock_ratio - weights

        # money - increase if sell, decrease if buy, no changes if hold
        prev_money = self.money_memory[agent][-1]
        curr_money = prev_money + sum(diff_stock_ratio * self.stock_volume_reference * self.data[agent]["Close"].values)
        self.money_memory[agent].append(curr_money)

        # total value - money + currently held stock value
        curr_total = curr_money + sum(weights * self.stock_volume_reference * self.data[agent]["Close"].values)
        self.total_value_memory[agent].append(curr_total)

        # actions memory
        self.actions_memory[agent].append(weights)
        last_day_memory = self.data[agent]

        # load next state
        self.day[agent] += 1
        self.data[agent] = self.df.loc[self.day[agent],:]
        self.covs[agent] = [[x[0][0] for x in self.data[agent]['cov_list']]]
        self.state[agent] =  np.append(np.array(self.covs[agent]), [self.data[agent][tech].values.tolist() for tech in self.tech_indicator_list ], axis=0)
        
        # calculate portfolio return
        # individual stocks' return * weight
        # todo: to fix to our version?
        portfolio_return = sum(((self.data[agent]["Close"].values / last_day_memory["Close"].values)-1)*weights)
        
        # update portfolio value
        # todo: to fix to our version?
        new_portfolio_value = self.portfolio_value[agent]*(1+portfolio_return)
        self.portfolio_value[agent] = new_portfolio_value

        # save into memory
        self.portfolio_return_memory[agent].append(portfolio_return)
        self.date_memory[agent].append(self.data[agent]["Date"].unique()[0])            
        self.asset_memory[agent].append(new_portfolio_value)

        # the reward is the new portfolio value or end portfolio value
        self.reward[agent] = new_portfolio_value 
        #self.reward = self.reward*self.reward_scaling
      
      # penalise or reward the target agent based on the result of all other agents
      all_rewards = list(self.reward.values())
      for agent in self.possible_agents:
        # ratio = current agent / other agent
        # if reward of current agent > other agent, ratio > 1, reward is increased
        # else ratio < 1, reward is penalised
        self.reward[agent] *= prod(self.reward[agent] / all_rewards)
        # self.reward[self.training_agent] *= (self.reward[self.training_agent] / pv)

    return self.state[agent], self.reward[self.training_agent], self.terminal, {}

  def reset(self, seed=None, return_info=False, options=None):
    # print("reset")

    # get glob values
    if not a2c_glob_init_pv:
      for agent in self.possible_agents:
        a2c_glob_init_pv[agent] = self.total_value_memory[agent]
        # a2c_glob_init_sharpe[agent] = self.total_value_memory[agent]

    # agents
    self.agents = self.possible_agents[:]

    # attributes
    self.day = {
        agent: 0 for agent in self.possible_agents
    }
    self.data = {
        agent: self.df.loc[self.day[agent],:] for agent in self.possible_agents
    }
    self.covs = {
        agent: [[x[0][0] for x in self.data[agent]['cov_list']]] for agent in self.possible_agents
    }
    self.state = {
        agent: np.append(np.array(self.covs[agent]), [self.data[agent][tech].values.tolist() for tech in self.tech_indicator_list ], axis=0) for agent in self.possible_agents
    }

    # memory
    self.portfolio_value = {
        agent: self.initial_amount for agent in self.possible_agents
    }
    self.asset_memory = {
        agent: [self.initial_amount] for agent in self.possible_agents
    }
    self.portfolio_return_memory = {
        agent: [0] for agent in self.possible_agents
    }
    # stock ratio
    self.actions_memory = {
        agent: [[0]*self.stock_dim] for agent in self.possible_agents
    }
    self.date_memory = {
        agent: [self.data[agent]["Date"].unique()[0]] for agent in self.possible_agents
    }

    # free cash
    self.money_memory = {
        agent: [self.initial_amount] for agent in self.possible_agents
    }
    # cash + stock value
    self.total_value_memory = {
        agent: [self.initial_amount] for agent in self.possible_agents
    }

    # rewards
    self.reward = {
        agent: None for agent in self.possible_agents
    }
    
    # misc
    self.terminal = False 
    #self.cost = 0
    #self.trades = 0
    
    return self.state[self.training_agent] 

  def render(self):
    return self.state[self.training_agent]

  def seed(self, seed=None):
    self.np_random, seed = seeding.np_random(seed)
    return [seed]
  
  def softmax_normalization(self, actions):
    numerator = np.exp(actions)
    denominator = np.sum(np.exp(actions))
    softmax_output = numerator/denominator
    return softmax_output

  def set_training_agent(self, agent):
    # print(agent)
    self.training_agent = agent

  def learn(self, total_timesteps=1000):
    # run till terminal in each timestep
    for n in range(total_timesteps):
      print("Step:", n)
      self.collect_individual_preds()
      for agent in self.possible_agents:
        self.set_training_agent(agent)
        self.agent_name_mapping[agent] = self.agent_name_mapping[agent].learn(total_timesteps=1)
        
      if n == 0:
        # save init for plot
        for agent in self.possible_agents:
          a2c_glob_init_pv[agent] = self.total_value_memory[agent]
    
    # save final for plot
    for agent in self.possible_agents:
      a2c_glob_pv[agent] = self.total_value_memory[agent]


In [ ]:
# configurations

stock_dimension = len(mixed_df["tic"].unique())
state_space = 2

env_kwargs = {
  "hmax": 100, 
  "initial_amount": 1000000, 
  "transaction_cost_pct": 0.001, 
  "state_space": state_space, 
  "stock_dim": stock_dimension, 
  "tech_indicator_list": [
    "volume_obv",
    "volume_adi", 
    "trend_adx", 
    "momentum_ao", 
    "trend_macd", 
    "momentum_rsi", 
    "momentum_stoch"
  ], 
  "action_space": stock_dimension, 
  "reward_scaling": 1e-4,
    
}

### A2C

In [ ]:
a2c_glob_init_pv = {}
a2c_glob_init_sharpe = {}

a2c_glob_pv = {}
a2c_glob_sharpe = {}

In [ ]:
comp_env = CompetitivePMEnv(df=mixed_df, algo_type="A2C", **env_kwargs)
comp_env.learn(
    total_timesteps=10,
)

# Save Past Data as JSON

In [ ]:
with open("data_init_1.json", "w") as data_file:
  json.dump(a2c_glob_init_pv, data_file)

with open("data_final_1.json", "w") as data_file:
  json.dump(a2c_glob_pv, data_file)

# Plots

## Market Trend

In [ ]:
plt.plot(stocks["AAPL"]["Close"].index, stocks["AAPL"]["Close"], label="AAPL")
plt.plot(stocks["TSLA"]["Close"].index, stocks["TSLA"]["Close"], label="TSLA")
plt.title("Daily Closing Value")
plt.legend()
plt.show()

## Portfolio Value

In [ ]:
for agent in comp_env.possible_agents:
  plt.plot(range(0, comp_env.end_day+1), a2c_glob_init_pv[agent], label=agent)
plt.title("Initial-timestep Portfolio Value")
plt.legend()
plt.show()

In [ ]:
for agent in comp_env.possible_agents:
  plt.plot(range(0, comp_env.end_day+1), a2c_glob_pv[agent], label=agent)
plt.title("Final-timestep Portfolio Value")
plt.legend()
plt.show()